In [ ]:
!pip install mlxtend
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/content/OnlineRetail.csv", encoding='ISO-8859-1')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [ ]:
df.shape

(541909, 8)

In [ ]:
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
8831,537153,22566,FELTCRAFT HAIRBAND PINK AND PURPLE,1,12/5/2010 13:03,0.85,16718.0,United Kingdom
406152,571737,20727,LUNCH BAG BLACK SKULL.,10,10/19/2011 10:31,1.65,14367.0,United Kingdom
61799,541485,21447,12 IVORY ROSE PEG PLACE SETTINGS,1,1/18/2011 13:05,1.25,18118.0,United Kingdom
155085,549964,71053,WHITE METAL LANTERN,4,4/13/2011 14:12,3.75,13122.0,United Kingdom
390173,570499,21217,RED RETROSPOT ROUND CAKE TINS,1,10/11/2011 9:19,9.95,12772.0,Portugal


In [ ]:
df.dtypes

,0
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,object
UnitPrice,float64
CustomerID,float64
Country,object


In [ ]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [ ]:
df.dropna(axis=0, subset=['Description'], inplace=True)

In [ ]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,133626
Country,0


In [ ]:
import re #regular expressions

df['Description'] = df['Description'].astype('str').apply(lambda x: re.sub(r'\s+', ' ',x).strip())


In [ ]:
df[df['InvoiceNo'].astype(str).str.contains('C')].sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
70677,C542094,22236,CAKE STAND 3 TIER MAGIC GARDEN,-1,1/25/2011 12:47,12.75,13310.0,United Kingdom
217762,C555935,23232,WRAP VINTAGE PETALS DESIGN,-25,6/8/2011 9:14,0.42,12567.0,France
85408,C543469,22503,CABIN BAG VINTAGE PAISLEY,-1,2/8/2011 14:55,12.75,12709.0,Germany
17987,C537788,16202E,BLACK PHOTO ALBUM,-1,12/8/2010 12:49,5.55,15916.0,United Kingdom
82287,C543195,22423,REGENCY CAKESTAND 3 TIER,-1,2/4/2011 12:29,10.95,13113.0,United Kingdom
134564,C547856,47566,PARTY BUNTING,-4,3/27/2011 12:14,4.95,16256.0,United Kingdom
400730,C571346,23295,SET OF 12 MINI LOAF BAKING CASES,-6,10/17/2011 11:46,0.83,16101.0,United Kingdom
72414,C542245,22380,TOY TIDY SPACEBOY,-1,1/26/2011 15:06,2.10,13098.0,United Kingdom
268310,C560409,84949,SILVER HANGING T-LIGHT HOLDER,-6,7/18/2011 14:24,1.65,16717.0,United Kingdom
11521,C537251,22466,FAIRY TALE COTTAGE NIGHTLIGHT,-3,12/6/2010 10:45,1.95,NaN,United Kingdom


In [ ]:
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
print(df.shape)

(531167, 8)


In [ ]:
print("Total number of countries:", df['Country'].nunique(), end="\n\n")
df['Country'].value_counts(normalize=True).head()

Total number of countries: 38



,proportion
Country,
United Kingdom,0.915283
Germany,0.017023
France,0.015829
EIRE,0.014862
Spain,0.004678


In [ ]:
grouped_df = (df[df['Country'] == "Germany"]
              .groupby(['InvoiceNo', 'Description'])
              .size() #count
              .unstack() #pivot method
              .fillna(0)
              .reset_index()
              .set_index('InvoiceNo'))
print(grouped_df.shape)
grouped_df.head()

(457, 1693)


Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC STAR T-LIGHT HOLDER,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def custom_encoding(x):
  if x <= 0:
    return 0
  if x >= 1:
    return 1

grouped_df = grouped_df.applymap(custom_encoding)
grouped_df.drop('POSTAGE', inplace=True, axis=1)

In [ ]:
frequent_itemsets = apriori(grouped_df, min_support=0.05, use_colnames=True)

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6, support_only=False, num_itemsets=None)

In [ ]:
rules.sort_values(by = 'lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,1.0,0.050194,5.587746,0.913551,0.428571,0.821037,0.654634
1,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,1.0,0.092679,4.529540,0.837922,0.483871,0.779227,0.684524
2,(SPACEBOY LUNCH BOX),(ROUND SNACK BOXES SET OF4 WOODLAND),0.102845,0.245077,0.070022,0.680851,2.778116,1.0,0.044817,2.365427,0.713415,0.251969,0.577243,0.483283
